# 重みパラメータとバイアス、そして学習

このノートブックでは、CNNの**重みパラメータ**と**バイアス**、そして**学習**の概念を理解します。

## 目次
1. 畳み込みフィルタのスライド（図3.9）
2. 重みパラメータとバイアスの表記（図3.10）
3. 予測と学習の違い（図3.11）
4. ReLU活性化関数
5. 具体的な計算式

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. 畳み込みフィルタのスライド（図3.9）

24×24の入力画像に対して、5×5のフィルタをstride=1で適用すると：

$$\text{出力サイズ} = \frac{24 - 5}{1} + 1 = 20$$

つまり、20×20の特徴マップが生成されます。

In [ ]:
# 図3.9: フィルタのスライドを視覚化

fig, axes = plt.subplots(1, 4, figsize=(14, 4))

input_size = 24
filter_size = 5

# 4つの位置でフィルタの移動を表示
positions = [(0, 0), (0, 5), (5, 0), (19, 19)]  # 左上、少し右、少し下、右下
titles = ['位置(0,0)\n左上', '位置(0,5)\n5ピクセル右', '位置(5,0)\n5ピクセル下', '位置(19,19)\n右下（最後）']

for idx, ((row, col), title) in enumerate(zip(positions, titles)):
    ax = axes[idx]
    
    # 入力画像（24×24）
    img = np.zeros((input_size, input_size))
    ax.imshow(img, cmap='gray', vmin=0, vmax=1)
    
    # フィルタの位置を示す赤枠
    from matplotlib.patches import Rectangle
    rect = Rectangle((col-0.5, row-0.5), filter_size, filter_size, 
                      fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
    
    ax.set_title(title, fontsize=10)
    ax.set_xlim(-0.5, input_size-0.5)
    ax.set_ylim(input_size-0.5, -0.5)
    ax.set_xticks([0, 23])
    ax.set_yticks([0, 23])

plt.suptitle('図3.9: 5×5フィルタが24×24画像上をスライド → 20×20の出力', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"入力サイズ: {input_size}×{input_size}")
print(f"フィルタサイズ: {filter_size}×{filter_size}")
print(f"出力サイズ: ({input_size}-{filter_size})/1+1 = {input_size - filter_size + 1}×{input_size - filter_size + 1}")

## 2. 重みパラメータとバイアスの表記（図3.10）

### 記号の意味

| 記号 | 意味 | 例 |
|-----|------|----|
| $x_{i,j}$ | 入力データのi行j列目 | $x_{2,3}$は2行3列目のピクセル値 |
| $w^k_{i,j}$ | フィルタk枚目のi行j列目の**重み** | $w^1_{2,3}$はフィルタ1枚目の2行3列目 |
| $c^k_{i,j}$ | 特徴マップk枚目のi行j列目の**出力** | $c^1_{2,3}$は特徴マップ1枚目の2行3列目 |
| $b_k$ | フィルタk枚目の**バイアス** | $b_1$はフィルタ1枚目のバイアス |

### パラメータ数の計算

5×5のフィルタを2枚使う場合：
- 重みパラメータ: $5 \times 5 \times 2 = 50$個
- バイアス: $2$個
- **合計: 52個**

In [ ]:
# パラメータ数の計算
filter_h, filter_w = 5, 5
num_filters = 2

weight_params = filter_h * filter_w * num_filters
bias_params = num_filters
total_params = weight_params + bias_params

print("=== 畳み込み層（1回目）のパラメータ数 ===")
print(f"フィルタサイズ: {filter_h}×{filter_w}")
print(f"フィルタ枚数: {num_filters}")
print(f"")
print(f"重みパラメータ: {filter_h}×{filter_w}×{num_filters} = {weight_params}個")
print(f"バイアス: {num_filters}個")
print(f"合計: {total_params}個")

## 3. 予測と学習の違い（図3.11）

一次関数 $y = ax + b$ で考えてみましょう。

| | 予測 | 学習 |
|--|------|------|
| 目的 | 入力xから出力yを計算 | パラメータa,bを最適化 |
| パラメータa,b | 固定 | 更新される |
| 使うデータ | 入力x | 入力x、予測値y、正解値 |

### 機械学習の本質
**学習** = パラメータを最適化して、予測値の精度を向上させること

In [ ]:
# 図3.11: 予測と学習の違いを視覚化

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 左: 予測
ax = axes[0]
x = np.linspace(0, 5, 100)
a, b = 1.5, 1  # 固定されたパラメータ
y = a * x + b
ax.plot(x, y, 'b-', linewidth=2, label=f'y = {a}x + {b}')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.scatter([2], [a*2+b], color='red', s=100, zorder=5)
ax.annotate(f'入力x=2 → 出力y={a*2+b}', xy=(2, a*2+b), xytext=(2.5, a*2+b+1),
            arrowprops=dict(arrowstyle='->', color='red'), fontsize=10)
ax.set_xlabel('x（入力）')
ax.set_ylabel('y（出力）')
ax.set_title('予測\nパラメータa,bは固定\n入力xから出力yを計算', fontsize=11)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.5, 5)
ax.set_ylim(-0.5, 10)

# 右: 学習
ax = axes[1]
# 正解データ（散布点）
np.random.seed(42)
x_data = np.array([1, 2, 3, 4])
y_true = 2 * x_data + 0.5 + np.random.randn(4) * 0.3
ax.scatter(x_data, y_true, color='green', s=100, label='正解データ', zorder=5)

# 学習前の予測線
ax.plot(x, 1.0 * x + 2, 'r--', linewidth=1, alpha=0.5, label='学習前: y=1.0x+2')
# 学習後の予測線
ax.plot(x, 2.0 * x + 0.5, 'b-', linewidth=2, label='学習後: y=2.0x+0.5')

ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.set_xlabel('x（入力）')
ax.set_ylabel('y（出力）')
ax.set_title('学習\n正解データを使って\nパラメータa,bを更新', fontsize=11)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.5, 5)
ax.set_ylim(-0.5, 10)

plt.suptitle('図3.11: AIによる「予測」と「学習」の違い', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. ReLU活性化関数

畳み込み演算の結果に適用される**活性化関数**です。

$$f(x) = \begin{cases} 0 & (x < 0) \\ x & (x \geq 0) \end{cases}$$

### 意味
- **負の値** → 0にカット
- **正の値** → そのまま通す

### なぜ必要？
- 非線形性を導入（線形変換だけでは複雑なパターンを学習できない）
- 計算が単純で高速

In [ ]:
# ReLU関数の実装と可視化

def relu(x):
    """ReLU活性化関数"""
    return np.maximum(0, x)

x = np.linspace(-5, 5, 100)
y = relu(x)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 左: ReLU関数のグラフ
ax = axes[0]
ax.plot(x, y, 'b-', linewidth=2)
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.fill_between(x[x<0], y[x<0], alpha=0.3, color='red', label='x<0 → 0')
ax.fill_between(x[x>=0], 0, y[x>=0], alpha=0.3, color='blue', label='x≥0 → x')
ax.set_xlabel('x（入力）')
ax.set_ylabel('f(x)（出力）')
ax.set_title('ReLU関数: f(x) = max(0, x)', fontsize=11)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(-5, 5)
ax.set_ylim(-1, 5)

# 右: 具体例
ax = axes[1]
examples = [-3, -1, 0, 2, 5]
results = [relu(e) for e in examples]
colors = ['red' if e < 0 else 'blue' for e in examples]

bars = ax.bar(range(len(examples)), results, color=colors, alpha=0.7)
ax.set_xticks(range(len(examples)))
ax.set_xticklabels([f'x={e}\n→{r}' for e, r in zip(examples, results)])
ax.set_ylabel('ReLU(x)')
ax.set_title('ReLUの計算例', fontsize=11)
ax.axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print("ReLU関数の計算例:")
for e in examples:
    print(f"  ReLU({e:2d}) = {relu(e)}")

## 5. 具体的な計算式

特徴マップの出力 $c^1_{1,1}$ は以下のように計算されます：

$$c^1_{1,1} = f\left(\sum_{i=1}^{5}\sum_{j=1}^{5} w^1_{i,j} \cdot x_{i,j} + b_1\right)$$

展開すると：
$$c^1_{1,1} = f(w^1_{1,1}x_{1,1} + w^1_{1,2}x_{1,2} + \cdots + w^1_{5,5}x_{5,5} + b_1)$$

つまり：
1. 5×5=25個の積和演算
2. バイアス$b_1$を加算
3. ReLU関数$f$を適用

In [ ]:
# 具体的な計算例

# ランダムな入力データ（5×5領域）
np.random.seed(42)
x = np.random.randint(0, 10, (5, 5))

# ランダムな重みパラメータ
w = np.random.randn(5, 5).round(2)

# バイアス
b = 0.5

print("入力データ x (5×5):")
print(x)
print("\n重みパラメータ w (5×5):")
print(w)
print(f"\nバイアス b = {b}")

# 積和演算
dot_product = np.sum(x * w)
print(f"\n積和演算: Σ(x * w) = {dot_product:.2f}")

# バイアス加算
with_bias = dot_product + b
print(f"バイアス加算: {dot_product:.2f} + {b} = {with_bias:.2f}")

# ReLU適用
output = relu(with_bias)
print(f"ReLU適用: f({with_bias:.2f}) = {output:.2f}")
print(f"\n最終出力 c = {output:.2f}")

In [ ]:
# PyTorchでの実装例
import torch
import torch.nn as nn

# 畳み込み層 + ReLU
conv_relu = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=1, bias=True),
    nn.ReLU()
)

# パラメータ数を確認
total_params = sum(p.numel() for p in conv_relu.parameters())
print(f"総パラメータ数: {total_params}")
print(f"  - 重み: 5×5×1×2 = {5*5*1*2}")
print(f"  - バイアス: 2")

# 24×24の入力に適用
dummy_input = torch.randn(1, 1, 24, 24)
output = conv_relu(dummy_input)
print(f"\n入力サイズ: {dummy_input.shape}")
print(f"出力サイズ: {output.shape}  # (batch, channels, height, width)")

## まとめ

### 重みパラメータとバイアス
- **重みパラメータ $w$**: フィルタの各要素の値（学習で更新される）
- **バイアス $b$**: 積和演算結果に加算する定数（学習で更新される）
- パラメータ数 = フィルタサイズ × フィルタ枚数 + バイアス数

### 予測と学習
- **予測**: パラメータ固定で入力から出力を計算
- **学習**: 誤差を最小化するようにパラメータを更新

### ReLU活性化関数
- $f(x) = \max(0, x)$
- 負の値を0にカット、正の値はそのまま
- 非線形性を導入

## 次のステップ

次のノートブックでは、**誤差逆伝播法**による学習のメカニズムを学びます。